### Load S&P 500 component and SPMO data:

In [35]:
import pandas as pd
import numpy as np
from pypfopt import expected_returns, risk_models, EfficientFrontier

### Load market data and SPMO data:

In [39]:
# Load data from CSV files
spx_data = pd.read_csv('SPX_data.csv', index_col='date', parse_dates=True)
spmo_data = pd.read_csv('SPMO_data.csv', parse_dates=True)

# Process SPMO data
spmo_data['date'] = pd.to_datetime(spmo_data['date'])
spmo_data = spmo_data.set_index('date')
spmo_data = spmo_data[spmo_data['Ticker'] == 'SPMO-USA']['Price']
spmo_data.name = 'SPMO'

# Combine SPX and SPMO data
combined_df = pd.concat([spx_data, spmo_data], axis=1)

# Ensure all dates match by dropping any rows with missing data
combined_df = combined_df.dropna()

# Split the combined dataframe back into market returns and SPMO returns
aligned_market_returns = combined_df.drop('SPMO', axis=1)
aligned_spmo_returns = combined_df['SPMO']

In [40]:
# Calculate 12-month returns for market and SPMO
aligned_market_returns_12m = aligned_market_returns.pct_change(periods=12).dropna()
aligned_spmo_returns_12m = aligned_spmo_returns.pct_change(periods=12).dropna()

# Ensure both dataframes have the same index after calculating returns
common_index = aligned_market_returns_12m.index.intersection(aligned_spmo_returns_12m.index)
aligned_market_returns_12m = aligned_market_returns_12m.loc[common_index]
aligned_spmo_returns_12m = aligned_spmo_returns_12m.loc[common_index]

In [41]:
# Ensure prices are sorted by date
market_data_pivot = market_data_pivot.sort_index()

# Function to calculate 12-month returns excluding the most recent month
def calculate_12_month_returns(prices):
    if len(prices) < 13:
        return pd.Series(dtype=float)
    return (prices.shift(1) / prices.shift(13)) - 1

# Calculate 12-month returns for market data
market_returns_12m = market_data_pivot.apply(calculate_12_month_returns, axis=0)

# Drop rows where all columns have NaN values
market_returns_12m = market_returns_12m.dropna(how='all')

# Calculate 12-month returns for SPMO
spmo_returns_12m = calculate_12_month_returns(spmo_prices).dropna()

In [42]:
print(f"Market Returns Shape: {market_returns_12m.shape}")
print(f"SPMO Returns Shape: {spmo_returns_12m.shape}")

# Print the first few rows of each returns dataframe to verify
#print(market_returns_12m.head())
#print(spmo_returns_12m.head())

Market Returns Shape: (249, 503)
SPMO Returns Shape: (1030,)


In [43]:
# Print the length of the DataFrame before and after calculating returns
print(f"Original Market Data Length: {len(market_data_pivot)}")
print(f"Calculated Returns Length: {len(market_returns_12m)}")

# Print out specific columns or rows to debug
print(market_data_pivot.iloc[:, :5].head())  # Print the first 5 columns
print(market_returns_12m.iloc[:, :5].head())  # Print the first 5 columns of returns

Original Market Data Length: 262
Calculated Returns Length: 249
Ticker       A-USA  AAL-USA  AAPL-USA  ABBV-USA  ABNB-USA
date                                                     
2023-07-31  121.77    16.75   196.450    149.58    152.19
2023-08-01  122.48    16.23   195.605    148.54    148.91
2023-08-02  127.71    15.95   192.580    149.38    144.56
2023-08-03  125.05    15.98   191.170    149.05    140.88
2023-08-04  126.30    15.84   181.990    147.73    140.17
Ticker         A-USA   AAL-USA  AAPL-USA  ABBV-USA  ABNB-USA
date                                                        
2023-08-17 -0.003367 -0.078209 -0.101196  0.013170 -0.153492
2023-08-18 -0.013308 -0.072705 -0.110452  0.009964 -0.140488
2023-08-21 -0.066244 -0.057053 -0.093935  0.005088 -0.134892
2023-08-22 -0.049500 -0.051314 -0.080190  0.007447 -0.119818
2023-08-23 -0.068567 -0.063763 -0.026155  0.003452 -0.093387


In [44]:
# Verify the dates before and after processing
print(f"Market Data Dates: {market_data_pivot.index.min()} to {market_data_pivot.index.max()}")
print(f"Market Returns Dates: {market_returns_12m.index.min()} to {market_returns_12m.index.max()}")

# Check for missing dates
missing_dates = market_data_pivot.index.difference(market_returns_12m.index)
print(f"Missing Dates: {missing_dates}")

Market Data Dates: 2023-07-31 00:00:00 to 2024-07-30 00:00:00
Market Returns Dates: 2023-08-17 00:00:00 to 2024-07-30 00:00:00
Missing Dates: DatetimeIndex(['2023-07-31', '2023-08-01', '2023-08-02', '2023-08-03',
               '2023-08-04', '2023-08-07', '2023-08-08', '2023-08-09',
               '2023-08-10', '2023-08-11', '2023-08-14', '2023-08-15',
               '2023-08-16'],
              dtype='datetime64[ns]', name='date', freq=None)


In [32]:
# Ensure indexes are consistent
common_dates = market_returns_12m.index.intersection(spmo_returns_12m.index)
aligned_market_returns_12m = market_returns_12m.loc[common_dates]
aligned_spmo_returns_12m = spmo_returns_12m.loc[common_dates]

# Check the aligned dataframes
print(f"Aligned Market Returns Shape: {aligned_market_returns_12m.shape}")
print(f"Aligned SPMO Returns Shape: {aligned_spmo_returns_12m.shape}")
print(aligned_market_returns_12m.head())
print(aligned_spmo_returns_12m.head())

Aligned Market Returns Shape: (249, 503)
Aligned SPMO Returns Shape: (249,)
Ticker         A-USA   AAL-USA  AAPL-USA  ABBV-USA  ABNB-USA   ABT-USA  \
date                                                                     
2023-08-17 -0.003367 -0.078209 -0.101196  0.013170 -0.153492 -0.061080   
2023-08-18 -0.013308 -0.072705 -0.110452  0.009964 -0.140488 -0.050665   
2023-08-21 -0.066244 -0.057053 -0.093935  0.005088 -0.134892 -0.045203   
2023-08-22 -0.049500 -0.051314 -0.080190  0.007447 -0.119818 -0.034127   
2023-08-23 -0.068567 -0.063763 -0.026155  0.003452 -0.093387 -0.023223   

Ticker      ACGL-USA   ACN-USA  ADBE-USA   ADI-USA  ...   WTW-USA    WY-USA  \
date                                                ...                       
2023-08-17 -0.009525 -0.027533 -0.058004 -0.112063  ... -0.070459 -0.048738   
2023-08-18 -0.036446 -0.047761 -0.068166 -0.120263  ... -0.076199 -0.053282   
2023-08-21 -0.027404 -0.046699 -0.041807 -0.086823  ... -0.066710 -0.027108   
2023-08-22

In [47]:
# Convert to numeric, coercing errors to NaN
aligned_market_returns_12m = aligned_market_returns_12m.apply(pd.to_numeric, errors='coerce')

# Drop any remaining NaN values
aligned_market_returns_12m = aligned_market_returns_12m.dropna()

# Convert to float
aligned_market_returns_12m = aligned_market_returns_12m.astype(float)

### Use PyPortfolioOpt to minimize tracking error vs. SPMO index

In [48]:
import numpy as np
from pypfopt import expected_returns, risk_models, EfficientFrontier

def tracking_error(weights, market_returns, spmo_returns):
    # Compute the portfolio returns for each time period
    portfolio_returns = np.dot(market_returns, weights)
    
    # Compute the tracking error
    tracking_error_value = np.sqrt(np.mean((portfolio_returns - spmo_returns)**2))
    
    return tracking_error_value

# Calculate expected returns and sample covariance matrix
mu = expected_returns.mean_historical_return(aligned_market_returns_12m)
S = risk_models.sample_cov(aligned_market_returns_12m)

# Initialize the Efficient Frontier
ef = EfficientFrontier(mu, S)

# Define the tracking error function
def tracking_error_func(weights, target=None, weight=None):
    return tracking_error(weights, aligned_market_returns_12m, aligned_spmo_returns_12m)

# Add the custom objective to the Efficient Frontier
ef.add_objective(tracking_error_func)

# Optimize the portfolio
weights = ef.efficient_risk(target_risk=0.1)  # Adjust the target risk as needed

# Get the optimized weights
cleaned_weights = ef.clean_weights()

# Print the optimized weights
print("Optimized Portfolio Weights:")
for asset, weight in cleaned_weights.items():
    print(f"{asset}: {weight:.4f}")

# Calculate and print portfolio performance
performance = ef.portfolio_performance(verbose=True)

/home/codespace/.python/current/lib/python3.10/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/codespace/.python/current/lib/python3.10/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/home/codespace/.local/lib/python3.10/site-packages/pandas/core/frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
/home/codespace/.python/current/lib/python3.10/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/home/codespace/.python/current/lib/python3.10/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


ValueError: operands could not be broadcast together with shapes (0,3) (0,) 

In [ ]:
cleaned_weights = ef.clean_weights()
cleaned_weights

In [ ]:
performance = ef.portfolio_performance(verbose=True)
performance